## Script responsável por descompactar o ZIP

- Antes de executar esse script, fazer download da base do DOU no link: https://www.in.gov.br/acesso-a-informacao/dados-abertos/base-de-dados
- Há um arquivo zip para cada ano/mês/seção
- Salvar no subdiretório \downloads\Secao0<Número da secao>\ano<com 4 dígitos>. Exemplo: \downloads\Secao02\2024
- No processamento o script:
  - Descompacta todos os arquivos zips existentes no subdiretório downloads\Secao0<Número da secao>
  - Faz a leitura do XML
  - Cria a coluna Texto a partir da TextHTML, removento das tags html do corpo da portaria
  - Cria a coluna Ano com o ano da publicação
  - Cria a coluna Mês com o mês da publicação
  - Cria a coluna Total_palavras, que exibe o total de palavras da coluna Texto
  - Salva a saída no arquivo output9<seção>


In [8]:
import os
import zipfile
import xml.etree.ElementTree as ET
import pandas as pd
import io
from bs4 import BeautifulSoup
import string

In [2]:
douItem = 2
secao = f'Secao0{douItem}'

In [3]:
def process_zip(zip_path):

    print(zip_path)
    # Lista para armazenar os dados dos artigos
    articles_data = []

    # Abre o arquivo ZIP
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Itera sobre cada arquivo no ZIP
        for file_name in zip_ref.namelist():
            if file_name.endswith('.xml'):
                try:
                    # Abre o arquivo XML
                    with zip_ref.open(file_name) as file:
                        # Lê o conteúdo do arquivo
                        content = file.read()
                        # Verifica se o conteúdo não está vazio
                        if content.strip():
                            # Parse do conteúdo XML
                            tree = ET.ElementTree(ET.fromstring(content))
                            root = tree.getroot()

                            # Itera sobre todos os elementos <article>
                            for article in root.findall('article'):
                                if article is not None:
                                    # Extrai os atributos do <article>
                                    article_data = article.attrib

                                    # Extrai os conteúdos dos elementos dentro de <body>
                                    body = article.find('body')
                                    if body is not None:
                                        for child in body:
                                            # Usa o nome do elemento como chave e seu texto como valor
                                            article_data[child.tag] = child.text.strip() if child.text else None

                                    # Adiciona o nome do arquivo aos dados do artigo
                                    article_data['file_name'] = file_name
                                    # Adiciona o nome do arquivo ZIP aos dados do artigo
                                    article_data['zip_name'] = os.path.basename(zip_path)
                                    articles_data.append(article_data)
                except ET.ParseError:
                    print(f"Erro ao analisar XML no arquivo {file_name} no ZIP {zip_path}")
                except Exception as e:
                    print(f"Erro ao processar o arquivo {file_name} no ZIP {zip_path}: {e}")

    return articles_data

In [4]:
def process_all_zips_in_directory(directory):

    print(directory)
    # Lista para armazenar os dados de todos os artigos
    all_articles_data = []

    # Itera sobre todos os arquivos no diretório e subdiretórios
    for root, dirs, files in os.walk(directory):
        print(f'Processando diretório {root}...')
        for file_name in files:
            if file_name.endswith('.zip'):
                zip_path = os.path.join(root, file_name)
                articles_data = process_zip(zip_path)
                all_articles_data.extend(articles_data)

    # Cria um DataFrame a partir dos dados de todos os artigos
    df = pd.DataFrame(all_articles_data)
    return df

In [5]:
# Função para extrair o texto de um texto HTML
def extract_text_from_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    return soup.get_text(separator=' ')

In [6]:
# Função para contar palavras desconsiderando a pontuação
def contar_palavras(texto):
    # Remover pontuação
    texto_sem_pontuacao = texto.translate(str.maketrans('', '', string.punctuation))
    # Dividir o texto em palavras
    palavras = texto_sem_pontuacao.split()
    # Retornar a contagem de palavras
    return len(palavras)

In [9]:
douItem = input("Informe a seção: 1, 2 ou 3")
secao = f'Secao0{douItem}'

# Caminho para o diretório contendo os arquivos ZIP
directory_path = os.path.join('./downloads/', secao)

# Processa todos os ZIPs no diretório e obtém o DataFrame
df = process_all_zips_in_directory(directory_path)

# Renomeia a coluna Texto
df.rename(columns={'Texto': 'TextoHTML'}, inplace=True)

# Cria as colunas ano e mês
df['pubDate'] = pd.to_datetime(df['pubDate'], format='%d/%m/%Y', errors='coerce')
df = df[df['pubDate'].notna()]
# 2. Extrair ano e mês
df['Ano'] = df['pubDate'].dt.year
df['Mês'] = df['pubDate'].dt.month

# Cria uma coluna 'text_content' com o texto extraído do HTML
df['Texto'] = df['TextoHTML'].apply(extract_text_from_html)

# Cria uma coluna com o total de palavras da coluna Texto
df['Total_palavras'] = df['Texto'].apply(contar_palavras)

# Exibe o DataFrame
df.shape

Informe a seção: 1, 2 ou 3 2


./downloads/Secao02
Processando diretório ./downloads/Secao02...
Processando diretório ./downloads/Secao02\2024...
./downloads/Secao02\2024\S02012024.zip
./downloads/Secao02\2024\S02022024.zip
./downloads/Secao02\2024\S02032024.zip
./downloads/Secao02\2024\S02042024.zip
./downloads/Secao02\2024\S02052024.zip
./downloads/Secao02\2024\S02062024.zip
./downloads/Secao02\2024\S02072024.zip
./downloads/Secao02\2024\S02082024.zip
./downloads/Secao02\2024\S02092024.zip
./downloads/Secao02\2024\S02102024.zip


(149208, 31)

In [10]:
df.to_parquet(f'./saida/DOU{secao}.parquet', engine='pyarrow', index=False)

In [11]:
df.head()

,id,name,idOficio,pubName,artType,pubDate,artClass,artCategory,artSize,artNotes,...,Ementa,Titulo,SubTitulo,TextoHTML,file_name,zip_name,Ano,Mês,Texto,Total_palavras
0,35952390,PORTARIA GPR 2968,9975984,DO2,Portaria,2024-01-02,00070:00041:00002:00000:00000:00000:00000:0000...,Poder Judiciário/Tribunal de Justiça do Distri...,12,,...,None,None,None,"<p class=""identifica"">PORTARIA GPR Nº 2.968, D...",S02012024/529_20240102_21129548.xml,S02012024.zip,2024,1,"PORTARIA GPR Nº 2.968, DE 9 DE NOVEMBRO DE 202...",147
1,35951661,Leticia Sampaio Sarmet Moreira,10010756,DO2,Portaria,2024-01-02,00047:00034:00009:00008:00000:00000:00000:0000...,Ministério da Saúde/Secretaria de Atenção Espe...,12,,...,None,None,None,"<p class=""identifica"">Portaria Nº 283/MS/HFL, ...",S02012024/529_20240102_21177457.xml,S02012024.zip,2024,1,"Portaria Nº 283/MS/HFL, DE 22 DE NOVEMBRO DE 2...",142
2,35951832,PORTARIA NA 4468.23 - REITOR SUB,10022530,DO2,Portaria,2024-01-02,00025:00052:00038:00001:00000:00000:00000:0000...,Ministério da Educação/Instituto Federal de Ed...,12,,...,None,None,None,"<p class=""identifica"">PORTARIA Nº 4.468, DE 4 ...",S02012024/529_20240102_21193596.xml,S02012024.zip,2024,1,"PORTARIA Nº 4.468, DE 4 DE DEZEMBRO DE 2023 O ...",157
3,35952210,PORTARIA NA 4469.23 - REITOR SUB,10022530,DO2,Portaria,2024-01-02,00025:00052:00038:00001:00000:00000:00000:0000...,Ministério da Educação/Instituto Federal de Ed...,12,,...,None,None,None,"<p class=""identifica"">PORTARIA Nº 4.469, DE 4 ...",S02012024/529_20240102_21193597.xml,S02012024.zip,2024,1,"PORTARIA Nº 4.469, DE 4 DE DEZEMBRO DE 2023 O ...",156
4,35951670,Portaria NA 99 Aposentadoria Apo,10022818,DO2,Portaria,2024-01-02,00008:00002:00026:00000:00000:00000:00000:0000...,Ministério da Agricultura e Pecuária/Secretari...,12,,...,None,None,None,"<p class=""identifica"">Portaria Nº 99, 24 de ou...",S02012024/529_20240102_21193996.xml,S02012024.zip,2024,1,"Portaria Nº 99, 24 de outubro de 2023 O Superi...",150


In [12]:
totais = df.groupby('artType').size()
print(totais)

artType
Ato                            7640
Ato Concessório                 211
Circular                          1
Decisão                         222
Decreto de Pessoal              239
Deliberação                       9
Despacho                       7932
Edital                          685
Edital de Citação                56
Edital de Convocação             33
Edital de Intimação              18
Edital de Notificação           438
Lista de Antiguidade             13
Portaria                     127460
Portaria Conjunta               921
Portaria Interministerial        16
Processo Disciplinar              2
Provimento                        4
Recurso Disciplinar               2
Resolução                       331
Retificação                    2971
Retificação (de Edital)           4
dtype: int64


In [13]:
df_portaria = df[df['artType'] == 'Portaria']

df_portaria.shape

(127460, 31)

In [14]:
df_reduzido = df_portaria.head(1000)

# Supondo que 'df' seja o seu DataFrame
df_reduzido.to_excel(f'./saida/DOU{secao}_reduzido.xlsx', index=False)

In [15]:
# 3. Agrupar por ano e mês e contar as ocorrências
contagem_portarias = df_portaria.groupby(['Ano', 'Mês']).size().reset_index(name='Total de Portarias')
contagem_portarias.head(20)

,Ano,Mês,Total de Portarias
0,2024,1,12238
1,2024,2,12504
2,2024,3,12861
3,2024,4,13811
4,2024,5,12059
5,2024,6,11936
6,2024,7,14014
7,2024,8,12888
8,2024,9,12390
9,2024,10,12759
